An example of using "function score" feature in elasticsearch

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.client import CatClient
from elasticsearch.client import TasksClient

In [2]:
elastic_db = Elasticsearch(['localhost:9200'])
indices_client = IndicesClient(elastic_db)
cat_client = CatClient(elastic_db)
task_client = TasksClient(elastic_db)

In [3]:
# To view the indices on Elastisearch:
print(cat_client.indices(v=True))

NameError: name 'cat_client' is not defined

In [ ]:
# Create the dest_idx with required settings and mappings
idx_structure = {
    "settings":{
    },
    "mappings": {
        "doc": {
            "properties": {
                "sku":{"type":"text"},
                "productFamily":{"type":"text"},
                "price":{"type":"integer"},
                "vcpu":{"type":"integer"},
                "memory":{"type":"integer"}
            }
        }
    }
}

response = indices_client.create(index='fscore_idx',body=idx_structure)
print(response)

In [ ]:
# Add document 1
doc_body = {
    "sku": "sku1",
    "productFamily":"sku1_product",
    "price":25,
    "vcpu": 2,
    "memory":4
}

response = elastic_db.index(index='fscore_idx', doc_type='doc', body=doc_body,refresh=True)
print(response)

In [ ]:
# Add document 2
doc_body = {
    "sku": "sku2",
    "productFamily":"sku2_product",
    "price":50,
    "vcpu": 4,
    "memory": 8
}

response = elastic_db.index(index='fscore_idx', doc_type='doc', body=doc_body,refresh=True)
print(response)

In [ ]:
# Add document 3
doc_body = {
    "sku": "sku3",
    "productFamily":"sku3_product",
    "price":75,
    "vcpu": 8,
    "memory": 16 
}

response = elastic_db.index(index='fscore_idx', doc_type='doc', body=doc_body,refresh=True)
print(response)

In [ ]:
# Add document 4
doc_body = {
    "sku": "sku4",
    "productFamily":"sku4_product",
    "price":100,
    "vcpu": 16,
    "memory": 32
}

response = elastic_db.index(index='fscore_idx', doc_type='doc', body=doc_body,refresh=True)
print(response)

In [ ]:
# To view the indices on Elastisearch:
print(cat_client.indices(v=True))

In [ ]:
# QUERY FUNCTION. 
# Considers vcpu and memory field
# Other fields can be added
# Origin parameter will be the value to search
# offset/scale/decay parameters have to be set by trial and error depending on data and requirement
# "explain":True/False used to check the break up of score
sch_body = {
    "explain": True,
    "query":{
        "function_score":{
            "query":{
                "match_all":{}
            },
            "functions":[
                {
                    "gauss":{
                        "vcpu":{
                            "origin":"4",
                            "offset": "0.5",
                            "scale":"0.75",
                            "decay":"0.01"
                        }
                    }
                },
                {
                    "gauss":{
                        "memory":{
                            "origin":"7",
                            "offset": "0.5",
                            "scale":"0.75",
                            "decay":"0.0001"
                        }
                    }
                }
            ],
            "score_mode":"sum",
            "boost_mode": "replace"
        }
    }
}

response = elastic_db.search(index='fscore_idx', body=sch_body, explain=False)
print(response['hits']['hits'][0])